In [1]:
import numpy as np
import mlp_utils
from mlp_utils import Dense, ReLU, LogSoftMax, NNetwork

from importlib import reload

reload(mlp_utils)

<module 'mlp_utils' from '/home/whoisjiji/Desktop/crafted_neural_networks/mlp_utils.py'>

In [14]:
def iterate_minibatches(inputs, labels, batchsize, shuffle=True):
    if shuffle:
        indices = np.random.permutation(len(labels))
    else:
        indices = np.arange(len(labels))
    for start_idx in range(0, len(labels), batchsize):
        batch_indices = indices[start_idx : start_idx + batchsize]
        
        yield inputs[batch_indices], labels[batch_indices]

In [15]:
# Generating random data
np.random.seed(2)
dataset_size = 10**5
nclasses = 4
X = np.random.randn(dataset_size, 7*7)
Y = np.random.randint(low=0, high=nclasses-1, size=dataset_size)

In [16]:
LR = 0.01
input_size = np.prod(X.shape[1:]) ## skipping batch dim
model = NNetwork(layers = [
    Dense(input_size, 2**5, LR),
    ReLU(),
    Dense(2**5, 2**4, LR),
    ReLU(),
    Dense(2**4,nclasses, LR),
],
    loss = LogSoftMax()
)

In [17]:
model.layers[0].weights

array([[-0.21340989,  0.39631291, -0.86856653, ..., -1.29404792,
        -1.12570562,  1.29092966],
       [-0.23899517, -0.19442405,  2.20356364, ...,  1.4954862 ,
        -0.29619121, -0.60087935],
       [ 1.84882381,  0.83582694,  0.60435625, ...,  0.09464683,
         0.25997608, -1.61041033],
       ...,
       [ 1.75351663,  1.2794048 , -0.1313487 , ...,  1.08061383,
         0.89395252, -0.98280237],
       [-1.54338587,  0.56257521, -0.08125299, ..., -0.49492859,
        -0.83264532, -1.22570946],
       [ 0.75284819,  1.86935144, -1.14968957, ...,  0.10486069,
         0.0470952 ,  1.07654599]])

In [18]:
from IPython.display import clear_output
train_log = []
val_log = []

EPOCHS = 1
BATCHSIZE = 64

for epoch in range(EPOCHS):
    
    for x_batch, y_batch in iterate_minibatches(X, Y, BATCHSIZE):
        layer_inputs, logits = model.forward(x_batch) # FORWARD PASS
        
        
        # ENDING FORWARD (COMPUTING LOSS) + STARTING BACKWARD (COMPUTING LOGITS GRAD)
        batch_loss, grad_logits = model.compute_loss(logits, y_batch)
        
        break
        
  
        model.backward(layer_inputs, grad_logits)  # BACKWARD (UPDATING weights)
        
#         break

In [34]:
logits.max()

262.60057351515405

In [67]:
# # grad_output = grad_logits
# input, layer = list(reversed(list(zip(layer_inputs, model.layers))))[4]
# grad_output = layer.backward(input, grad_output)

3.5892453699373856

In [13]:
model.loss.forward(logits, y_batch)

102.09045600183748